## Imports

In [0]:
from functools import reduce
from pyspark.sql import functions as F
from pyspark.sql.functions import *

## Creat a list of tables

In [0]:
tableList = spark.sql("SHOW TABLES IN gov_br_combustivel.bronze_layer").select("tableName").collect()

## Unify tables in one dataframe

In [0]:
table_names = [row.tableName for row in tableList]

dfs = [
    spark.table(f"gov_br_combustivel.bronze_layer.{name}")
    for name in table_names
]

unified_df = dfs[0]
for df in dfs[1:]:
    unified_df = unified_df.unionByName(df, allowMissingColumns=True)

## Data Processing

## Drop null rolls

In [0]:
unified_df = unified_df.dropna(how="all")

### Null values

In [0]:
unified_df = unified_df.fillna("N/A", subset=['RegiaoSigla', 'EstadoSigla', 'Municipio', 'Revenda', 'NomeRua', 'NumeroRua', 'Complemento', 'Bairro', 'Cep', 'Produto', 'UnidadeMedida', 'CNPJRevenda', 'Bandeira'])
unified_df = unified_df.fillna("999.99", subset=["ValorCompra", "valorVenda"])

### Remove whitespace

In [0]:
unified_df = unified_df.withColumn("RegiaoSigla", trim(col("RegiaoSigla"))) \
    .withColumn("EstadoSigla", trim(col("EstadoSigla"))) \
    .withColumn("Municipio", trim(col("Municipio"))) \
    .withColumn("Revenda", trim(col("Revenda"))) \
    .withColumn("NomeRua", trim(col("NomeRua"))) \
    .withColumn("NumeroRua", trim(col("NumeroRua"))) \
    .withColumn("Complemento", trim(col("Complemento"))) \
    .withColumn("Bairro", trim(col("Bairro"))) \
    .withColumn("Cep", trim(col("Cep"))) \
    .withColumn("Produto", trim(col("Produto"))) \
    .withColumn("UnidadeMedida", trim(col("UnidadeMedida"))) \
    .withColumn("Bandeira", trim(col("Bandeira"))) \
    .withColumn("CNPJRevenda", trim(col("CNPJRevenda"))) \
    .withColumn("DataColeta", trim(col("DataColeta")))

### Table type menage

In [0]:
unified_df = unified_df.withColumn(
    "ValorVenda",
    F.regexp_replace(F.col("ValorVenda"), ",", ".").cast("double")
).withColumn(
    "DataColeta",
    F.to_date(F.col("DataColeta"), "dd/MM/yyyy")
)

## Removing default value from null columns

In [0]:
unified_df = unified_df.withColumn(
    "ValorVenda",
    F.when(F.col("ValorVenda") == 999.99, None).otherwise(F.col("ValorVenda"))
)

## Aggregation of Synonymous Terms and Name Correction

In [0]:
unified_df = unified_df.withColumn(
    "NumeroRua",
    F.when(F.col("NumeroRua") == "NA", "N/A").otherwise(F.col("NumeroRua"))
)

unified_df = unified_df.withColumn(
    "Complemento",
    F.when(F.col("Complemento") == "", "N/A").otherwise(F.col("Complemento"))
)

unified_df = unified_df.withColumn(
    "UnidadeMedida",
    F.when(F.col("UnidadeMedida") == "R$ / mÂ³", "R$ / m³").otherwise(F.col("UnidadeMedida"))
)

## Excluding Data Without Product

In [0]:
unified_df = unified_df.filter(F.col("Produto") != "N/A")

## Timestamp column to record silver treatment

In [0]:
unified_df = unified_df.withColumn("SilverDate", current_timestamp())

In [0]:
display(unified_df.schema)

## Save and optimize table

In [0]:
unified_df.write.format("delta") \
    .mode("overwrite") \
    .option("zorderBy", "CNPJRevenda") \
    .saveAsTable("gov_br_combustivel.silver_layer.unified_fuel_data")